In [2]:
import numpy as np
from pymongo import MongoClient
import pygad

import json
import time

In [3]:
def get_pokemon_individually():
    
    client = MongoClient()
    db = client['genetic_algo'] 

    # Individual pokemon's collection
    collec_indiv_pokemon = db['pokemons']
    pokemons = list(collec_indiv_pokemon.find())
    
    return pokemons

def get_pokemon_teams():
    
    client = MongoClient()
    db = client['genetic_algo'] 

    # pokemon's teams collection
    collec_poke_teams = db['pokemon_teams']
    collec_poke_teams = list(collec_poke_teams.find())
    
    for poke_team in collec_poke_teams:
        del poke_team['_id']    
    return collec_poke_teams

def best_against(pokeA, pokeB):
    
    best = 0
    pokemons_max_types = ['primaryType', 'secondaryType']
    
    for poke_typeA in pokemons_max_types:
        if poke_typeA in pokeA:

            for poke_typeB in pokemons_max_types:
                if poke_typeB in pokeB:
                    typeA_against_typeB = f'{poke_typeA}_against_{pokeB[poke_typeB]}'
                    if typeA_against_typeB in pokeA and pokeA[typeA_against_typeB] > best:

                        best = pokeA[typeA_against_typeB]
    return best



def battle(poke1, poke2):

    against1 = best_against(poke1, poke2)
    against2 = best_against(poke2, poke1)
    
    return poke1['CP'] * against1 - poke2['CP'] * against2

# Get all pokemons in a list
pokemons = get_pokemon_individually()





In [4]:
def fitness_func(ga_instance, solution, solution_idx):
    # Starts the fit value with 0
    fit = 0

    # Getting the rival team from the global variable
    global rival_team

    solution_team = [pokemons[solution_pokemon] for solution_pokemon in solution]
    for home_pokemon in solution_team:
        for rival_pokemon in rival_team:
            fit += battle(home_pokemon, rival_pokemon)

    return fit

fitness_function = fitness_func
parent_selection_type = 'rws'
crossover_type = "single_point"
mutation_type="random"

def run_genetic_algorithm():
    poke_rival_teams = get_pokemon_teams()[:25]
    results = dict()

    for rival_team_idx, rv_team_db in enumerate(poke_rival_teams):
        # Creating the global rival_team variable
        globals()['rival_team'] = rv_team_db[f'Team {rival_team_idx}'] 

        # Defining the fitnesse function
        fitness_function = fitness_func

        
        ga_instance = pygad.GA(num_generations= 200,
                               num_parents_mating=100 ,
                               fitness_func = fitness_function,
                               gene_type = int,
                               gene_space = np.arange(len(pokemons)),
                               sol_per_pop = 2000,
                               num_genes = 3,
                               parent_selection_type=parent_selection_type,
                               crossover_type=crossover_type,
                               crossover_probability = 0.8,
                               mutation_probability = 0.2,
                               mutation_type=mutation_type,
                               random_mutation_min_val = 0,
                               random_mutation_max_val = len(pokemons),
                               save_best_solutions = True)

        ga_instance.run()
        results[rival_team_idx] = dict(
                                       best_solutions= ga_instance.best_solutions.tolist(),
                                       best_solutions_fitness = ga_instance.best_solutions_fitness)
    
    return results

# Rodar o algoritmo
results = run_genetic_algorithm()


c:\Users\gabri\OneDrive\Área de Trabalho\Cursos\Pós_Grad\TCC\.venv\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
c:\Users\gabri\OneDrive\Área de Trabalho\Cursos\Pós_Grad\TCC\.venv\Lib\site-packages\pygad\pygad.py:1153: UserWarning: Use the 'save_best_solutions' parameter with caution as it may cause memory overflow when either the number of generations or number of genes is large.
  warnings.warn("Use the 'save_best_solutions' parameter with caution as it may cause memory overflow when either the number of generations or number o

In [5]:
saving_path = r'path'


with open(saving_path, 'w') as arquivo_json:
    json.dump(results, arquivo_json, indent = 4)